# Interaction of Parsed Data, Hashes, and Deployed Contracts
<hr/>

In [ ]:
# web3.../deploy.py
import json
from web3 import Web3
from dotenv import load_dotenv, find_dotenv
import os

# load from .env file
load_dotenv(find_dotenv())

import hashlib

# Constants
LOG_TXT_PATH = 'logfiles/workstationLog.txt'
DEVICE_XML_PATH = 'logfiles/Device.xml'
MAKO_TCW_PATH = 'logfiles/makoTest2.tcw'

### Run Loader Module

In [ ]:
%run ./loadContract.ipynb

### Interact with Smart Contract

In [ ]:
# Now we can interact with our contract with a call or a transact
# Call -> simulate making the call and getting the return value (no state change to blockchain)
# Transact -> actually make a state change (have to build & send transaction)
# Get latest transaction:
nonce = w3.eth.getTransactionCount(
    my_address
)  # gives our nonce - number of transactions
# initial value of hashNumber:
print(
    "Current value of favorite number: "
    + str(hash_storage.functions.retrieve().call())
)  # No state change!

print("Updating contract...")
# Store new value for hashNumber:
store_transaction = hash_storage.functions.store("0x8888").build_transaction(
    {
        "gasPrice": w3.eth.gas_price,
        "chainId": chain_id,
        "from": my_address,
        "nonce": nonce,
    }
)
signed_store_tx = w3.eth.account.sign_transaction(
    store_transaction, private_key=private_key
)
send_store_tx = w3.eth.send_raw_transaction(signed_store_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(send_store_tx)
print("Updated!")
print(
    "New value of hash: " + str(hash_storage.functions.retrieve().call())
)

## Query Blockchain and Log File to Compare Hashes

In [ ]:
# Read file in chunks (future-proofing) and generate hash:
def hashGenerator(file, buffer_size = 65536):
    """
    This function reads a given file in chunks and generates a corresponding
    SHA256 hash.

    Parameters
    ----------
    file : type 'str'
        relative path to file to hash
    buffer_size : {65536, 'other'}, optional
        number of bytes to read, by default 65536

    Returns
    -------
    file_hash
        the SHA256 hash of the file provided

    Raises
    ------
    N/A
    """
    file_hash = hashlib.sha256()
    # Read file as binary
    with open(file, 'rb') as f:
        chunk = f.read(buffer_size)
        # Keep reading and updating hash as long as there is more data:
        while len(chunk) > 0:
            file_hash.update(chunk)
            chunk = f.read(buffer_size)
    return file_hash

In [ ]:
# Function to update blockchain
def updateBlockChain(new_hash):
    """
    This function updates the blockchain by calling the 'store' function of the
    smart contract.

    Parameters
    ----------
    new_hash : type 'str'
        hash of file obtained from hashGenerator function

    Returns
    -------
    None

    Raises
    ------
    N/A
    """
    print("Updating contract...")

    # Get latest transaction:
    nonce = w3.eth.getTransactionCount(
        my_address
    )  # gives our nonce - number of transactions
    # Store new value for hashNumber:
    store_transaction = hash_storage.functions.store(new_hash).build_transaction(
        {
            "gasPrice": w3.eth.gas_price,
            "chainId": chain_id,
            "from": my_address,
            "nonce": nonce,
        }
    )
    signed_store_tx = w3.eth.account.sign_transaction(
        store_transaction, private_key=private_key
    )
    send_store_tx = w3.eth.send_raw_transaction(signed_store_tx.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(send_store_tx)
    print("Updated!")
    print(
        "New value of hash: " + str(hash_storage.functions.retrieve().call())
    )

In [ ]:
# Query the hash stored on the blockchain
on_chain_hash = str(hash_storage.functions.retrieve().call())
# Generate the hash of the log file
workstation_log_hash = '0x' + hashGenerator(LOG_TXT_PATH).hexdigest()
# compare the two - if different, update the blockchain!
if on_chain_hash != workstation_log_hash:
    updateBlockChain(workstation_log_hash)